In [1]:
import pandas as pd
import numpy as np
import shutil
import os


working_root="/mnt/w/6998/Project/data"


In [51]:
def clean_folder():
    try:
        shutil.rmtree(working_root)
    except OSError as e:
        print("Error: %s : %s" % (working_root, e.strerror))

    os.mkdir(working_root)
    os.mkdir(working_root+"-backup")

In [53]:
clean_folder()

Error: /mnt/w/6998/Project/data : No such file or directory


In [2]:
sample_num=10000
feature_num=10
class_num=2
dataset_num=2

for class_id in range(dataset_num):
    trainX=np.random.rand(sample_num,feature_num)
    for i in range(feature_num):
        scale=np.random.uniform(low=0,high=100)
        trainX[:,i]=trainX[:,i]*scale
    # trainY=np.random.randint(class_num,size=sample_num).reshape((-1,1))
    y=np.mean(trainX,axis=1)/100+0.7+0.05*class_id

    trainY=np.floor(y).reshape(-1,1)

    train_data=np.hstack((trainX,trainY))
    df=pd.DataFrame(train_data)
    col_names=[f"feature_{i+1}" for i in range(feature_num)]+[f"label{class_id}"]
    df.set_axis(col_names, axis=1,inplace=True)
    df.to_csv(working_root+f"-backup/date_{class_id}.csv", index=False)

In [3]:
click_log=pd.read_pickle('/mnt/w/6998/Project/Tencent2020_Rank1st/data/click.pkl')

In [4]:
click_log_drop_na_minus1 = click_log.loc[(click_log['age'] != -1) & (click_log['gender'] != -1) ] 
sample_log=click_log_drop_na_minus1.sample(n=450000)

click_log_drop_na_minus1_rev = click_log.loc[(click_log['age'] == -1) | (click_log['gender'] == -1) ] 
sample_log2=click_log_drop_na_minus1_rev.sample(n=50000)

sample_df=sample_log.append(sample_log2)

In [5]:

sample_df = sample_df.sample(frac=1).reset_index(drop=True)
filter_cols=list(filter(lambda cn: cn.startswith('age_') or cn.startswith('gender_'), sample_df.columns ))
print(filter_cols)
sample_df=sample_df.drop(filter_cols, axis = 1)

sample_df.loc[sample_df["age"]<5,"age"]=0
sample_df.loc[sample_df["age"]>=5,"age"]=1

sample_df.to_csv(working_root+"-backup/click_original.csv",index=False)

['age_0', 'age_1', 'age_2', 'age_3', 'age_4', 'age_5', 'age_6', 'age_7', 'age_8', 'age_9', 'gender_0', 'gender_1']


In [ ]:
sample_both=sample_df.iloc[:300000,:]
sample_gender=sample_df.iloc[300000:400000,:].drop(["age"],axis=1)
sample_age=sample_df.iloc[400000:,:].drop(["gender"],axis=1)
sample_both.to_csv(working_root+"-backup/click_300k_both.csv",index=False)
sample_gender.to_csv(working_root+"-backup/click_100k_gender.csv",index=False)
sample_age.to_csv(working_root+"-backup/click_100k_age.csv",index=False)

Another dataset

In [73]:

column_names = ['age', 'class_worker', 'det_ind_code', 'det_occ_code', 'education', 'wage_per_hour', 'hs_college',
                    'marital_stat', 'major_ind_code', 'major_occ_code', 'race', 'hisp_origin', 'sex', 'union_member',
                    'unemp_reason', 'full_or_part_emp', 'capital_gains', 'capital_losses', 'stock_dividends',
                    'tax_filer_stat', 'region_prev_res', 'state_prev_res', 'det_hh_fam_stat', 'det_hh_summ',
                    'instance_weight', 'mig_chg_msa', 'mig_chg_reg', 'mig_move_reg', 'mig_same', 'mig_prev_sunbelt',
                    'num_emp', 'fam_under_18', 'country_father', 'country_mother', 'country_self', 'citizenship',
                    'own_or_self', 'vet_question', 'vet_benefits', 'weeks_worked', 'year', 'income_50k']

# Load the dataset in Pandas
train_df = pd.read_csv(
    working_root+"-backup/census-income.data.gz",
    delimiter=',',
    header=None,
    index_col=None,
    names=column_names
)

train_df.loc[train_df["income_50k"]!=" 50000+.","income_50k"]=0
train_df.loc[train_df["income_50k"]==" 50000+.","income_50k"]=1

train_df.loc[train_df["marital_stat"]!=" Never married","marital_stat"]=1
train_df.loc[train_df["marital_stat"]==" Never married","marital_stat"]=0

label_columns = ['income_50k', 'marital_stat']
# One-hot encoding categorical columns
categorical_columns = ['class_worker', 'det_ind_code', 'det_occ_code', 'education', 'hs_college', 'major_ind_code',
                        'major_occ_code', 'race', 'hisp_origin', 'sex', 'union_member', 'unemp_reason',
                        'full_or_part_emp', 'tax_filer_stat', 'region_prev_res', 'state_prev_res', 'det_hh_fam_stat',
                        'det_hh_summ', 'mig_chg_msa', 'mig_chg_reg', 'mig_move_reg', 'mig_same', 'mig_prev_sunbelt',
                        'fam_under_18', 'country_father', 'country_mother', 'country_self', 'citizenship',
                        'vet_question']
transformed_train = pd.get_dummies(train_df, columns=categorical_columns)

    
print(transformed_train)    
    
other_df = pd.read_csv(
    working_root+'-backup/census-income.test.gz',
    delimiter=',',
    header=None,
    index_col=None,
    names=column_names
)

# First group of tasks according to the paper
label_columns = ['income_50k', 'marital_stat']

        age  wage_per_hour  capital_gains  capital_losses  stock_dividends  \
0        73              0              0               0                0   
1        58              0              0               0                0   
2        18              0              0               0                0   
3         9              0              0               0                0   
4        10              0              0               0                0   
...     ...            ...            ...             ...              ...   
199518   87              0              0               0                0   
199519   65              0           6418               0                9   
199520   47              0              0               0              157   
199521   16              0              0               0                0   
199522   32              0              0               0                0   

        instance_weight  num_emp  own_or_self  vet_benefits  we

In [72]:
train_df.to_csv(working_root+"-backup/census-income.csv",index=False)

In [74]:
transformed_train.to_csv(working_root+"-backup/census-incom22e.csv",index=False)